## Reexecução

In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.environ['SNORKELDB'] = 'postgres:///lzirondi'
from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
relation = candidate_subclass('Causa', ['Substância', 'Evento_Adverso'])

## Criando os arquivos dos dicionários

In [3]:
import Preprocessing as prep

inPath = None
outPath = None
corpusPath = None
LIWCpath = '/home/lzirondi/Github/snorkel/Scripts/'

util = prep.Util(inPath, outPath, LIWCpath, False)

util.LIWKdicios()

Dicionário LIWC de Causas gerado com sucesso.
Dicionário LIWC de Verbos gerado com sucesso.
Dicionário LIWC de Causas & Verbos gerado com sucesso.


## Criando os dicionários

In [18]:
paths = ['/home/lzirondi/Github/snorkel/Scripts/Dicts/Causas',
         '/home/lzirondi/Github/snorkel/Scripts/Dicts/Verbos',
         '/home/lzirondi/Github/snorkel/Scripts/Dicts/Causas & Verbos'
]

causas = {}
verbos = {}
causas_e_verbos = {}

dicios = {
    paths[0] : causas,
    paths[1] : verbos,
    paths[2] : causas_e_verbos
}

for file in paths:
    f = open(file, 'r', encoding='UTF-8')
    s = f.readlines()
    f.close()
    
    for word in s:
        word = word.replace('\n', '')
        dicios.get(file)[word] = 0    

## Análise

In [ ]:
from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)

from nltk.corpus import stopwords

cands = session.query(relation).filter(relation.split == 0).all()

#chegar no candidato e ver se entre os tokens tem algo que a gente quer

candidate = cands[0]
s = set()

for word in get_between_tokens(candidate):
    s.add(word)
    
sw = set(stopwords.words("portuguese"))

print(s)
s.difference_update(sw) 
print(s)

for word in get_between_tokens(candidate):
    if word not in sw:
        
    
    



## Testes

In [11]:
lista = [1,2,3]

dicio = {}

dicios = {}

d = {
    lista[0] : dicio,
    lista[1] : dicios
}


print(d)

{1: {}, 2: {}}
